#last modified (yyyy/mm/dd): 2024/08/22

Author: Alessandro Ulivi (ale.ulivi@gmail.com)

The present pipeline allows to perform a semantic segmentation of 1 channel. It was conceptualized to segment C. elegans embryo, imaged at the cortical plane, from background pixels.

When the pipeline was developed the embryo was imaged between the 1 and the 6 cells stage. C. elegans cortex was labelled with fluorescent dyes (fusion proteins) and imaged by means of microscopy techniques for fluorescence imaging. Precisely, the pipeline have been developed and tested on time-lapse images acquired, live, at a Nikon CSU-X1 spinning disk microscope at IGMBC Strasbourg, using a 100x, 1.4 NA, oil immersion objective (pixel size xy 0.11 um).

The pipeline works on 3D arrays. It was conceptualized for processing a single plane (the embryonic cortical plane) imaged over time. Thus, the segmentation masks are obtained by iterating 2D segmentations methods along one of the 3 axes of the input arrays. When the pipeline was created this corresponded to the time-dimension of the time-lapses.

To properly work the pipeline requires an existing input folder (input_folder) containg a single 3D arrays to process. It is not possible to have additional files in the input folder. The iteration axis must be in position 0 of the 3D array to process. When the pipeline was created this corresponded to the time-axis and the inputs had dimensions TYX.

It is required to provide an output folder. The output folder must exist. The output folder can contain files. The outputs of the pipeline are saved in the output folder.

The output of the pipeline are:
1) A binary segmentation mask of the same shape of the input 3D array. By default the output dtype is np.uin8 and values 0 and 255.

It is possible to restrict the segmentation to one or more regions of interest (ROIs) of the input 3D array. In this case, the ROIs must be provided as a single file (.roi or .zip) contained in a folder. Nothing else but the file must be present in the folder. The directory of the folder must be indicated in the variable roi_path. If the ROI should be used for segmentation the variable use_roi_for_segmentation must be set to True. The procedure is conceptualized for the use of ROI files created in ImageJ/Fiji.


The following cell import packages and functions required for the pipeline.

The cell should be run.

The cell should not be modified.

In [1]:
#Import packages
import os
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from utils import listdirNHF, form_mask_from_roi
from intensity_hist_based_embryo_segmentation import mask_embryo



DEFINE INPUT AND OUTPUT FOLDERS - DEFINE COMMON VARIABLES FOR DIFFERENT PROCESSING PARTS - OPEN THE INPUT FILE

The following cell must be run.

Some parts of the following cell must be changed according to the processing to do.

In [2]:
#indicate the directories of the input_folder and the output_folder.
input_folder = r""
output_folder = r""

#indicate saving names for segmentation mask - indicate the name without extension
mask_sav_ing_name = "embryo_mask"

#indicate in which dimension to iterate the 2D segmentation methods - when the pipeline was created this axis corresponded to time in the time-lapse image - when input files are created using
#open_rearrange_nikon_files this dimension is 0
time_axis = 0

#=== OPEN THE INPUT FILE - CREATE THE PREFIX FOR THE SAVING NAME === DON'T MODIFY THE LINES BELOW
#Open files in the input_folder as a list
input_files_l = listdirNHF(input_folder)

#Get file extension
extension = ".tif"
for e in ['.TIF', '.ome.tif']:
    if e in input_files_l[0]:
        extension=e

#Get the part of the file name before the extension and use it as the initial part of the saving name
extension_index = input_files_l[0].index(extension)
prefix_name = input_files_l[0][:extension_index]

#Open the input file (the single file in input_folder)
ch1_timecourse = tifffile.imread(os.path.join(input_folder,input_files_l[0]))


SEGMENTATION OF EMBRYO CORTEX

Run the following 2 cells to obtain and save a binary segmentation of the time-lapse. When the pipeline was created this segmentation allowed to segment the embryo cortex from the backgound.

To do the segmentation, the next two cells must be run. The first of the next two cells could be modified. The second of the next two cells should not be modified.





The segmentation follows the following steps:
1) Each 2D array (i) along the iteration axis of input array is smoothed using a gaussian kernel. The size of the pixel kernel can be specified in the variable gaussian_smoothing_kernel below. Output name GAU(i).
2) The histogram distribution of intensity values is calculated (bins=100) per each GAU(i) along the iteration axis of input array. Output name HIS(i).
3) Per each HIS(i) along the iteration axis of input array, the position on the x axis of the mode value for the histogram distribution is calculated. Output name MOD(i).
4) Per each HIS(i) along the iteration axis of input array, an intensity value IV1(i) is calculated. IV1(i) corresponds to the minimum of the HIS(i) when only values higher than MOD(i) are considered.
5) Per each HIS(i) along the iteration axis of input array, an intensity value IV2(i) is calculated. IV2(i) corresponds to the maximum of HIS(i) when only values higher than MOD(i) are considered.
6) Per each HIS(i) along the iteration axis of input array, an intensity value IV3(i) is calculated. IV3(i) corresponds to the minimum of HIS(i) when only values higher than MOD(i) and smaller than IV2(i) are considered.
7) IV1 and IV2 are pooled for all 2D arrays along the iteraton axis. Output name POOL.
8) The median M and standard deviation S of POOL are calculated.
9) An intensity value THRESHOLD(i) is calculated per each 2D array (i) along the iteration axis of input array. THRESHOLD(i) corresponds to IV1(i) if IV1(i) is within 1 S from M. Otherwise, THRESHOLD(i) corresponds to IV2(i) if IV2(i) is within 1 S from M. Otherwise, THRESHOLD(i) corresponds to M.
10) Each gaussian smoothed 2D array GAU(i) along the iteraton axis is binarized using THRESHOLD(i) as highpass filter. Pixels whose intensity values are >THRESHOLD(i) are set to positive  Output THRESH_IMG(i).
11) Individual regions of THRESH_IMG(i) are groups of connected pixels with the same intensity value and completely sourranded by backgroung pixels. Refer to https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.label . Individual regions of each 2D THRESH_IMG(i) along the iteration axis are filered using a highpass filter. The value of the highpass filter (in number of pixels) is defined in the variable embryo_highpass_area_threshold. Only regions whose area is >embryo_highpass_area_threshold are maintained. Output FINAL_THRESH_IMG(i).


In [3]:
#THESE PARAMETERS COULD BE CHANGED TO REFINE THE SEGMENTATION

#Indicate the size of the kernel used to smooth images before calculating the binarization threshold - high values result in smoother by less precise segmentation masks 
gaussian_smoothing_kernel = 5 #Examples: value was set to 5 when embryo cortex was segmented using pip2cherry labelling. It was set to 20 when the embryo cortex was segmented using cyk1gfp labelling.

#Highpass filter area - most likely the following line should not be modified
embryo_highpass_area_threshold = 250 #Segmented structures smaller than this value (the unit is number of pixels) are removed. Suggested value is 250.

#Indicate if an an roi should be used for the embryo segmentation
#NOTE:
# 1) the roi is meant to restrict the segmentation procedure to a part of the time-course. This means that regions outside the provided roi will be automatically
# excluded from the segmentation result. This could be useful if, for example, there are bright structures near the target segmentation structure (supposedly the embryo cortex)
# which might interfere with the segmentation.
# 2) if use_roi_for_segmentation is set to True, the directory of the folder where the roi is stored must be indicated in roi_path
use_roi_for_segmentation = False
roi_path = r"" #Ignore this line if use_roi_for_segmentation=False



In [4]:
#GET (EMBRYO CORTEX) SEGMENTATION MASK FOR ALL TIMEPOINTS - THE PRESENT CELL SHOULD NOT BE MODIFIED

#Get the roi mask for segmentation if provided
if use_roi_for_segmentation:
    embryo_roi_file_path = os.path.join(roi_path, listdirNHF(roi_path)[0])
    embryo_roi_mask = form_mask_from_roi(embryo_roi_file_path, ch1_timecourse[0,...]) #This is the reason why the iteration axis must be in position 0 of the input array


# #Use mask_embryo on ch2 (pip2-mCherry, cortex plane) to get the mask of the embryo.
if use_roi_for_segmentation:
    embryo_thresholded_timecourse = mask_embryo(ch1_timecourse,
                                                area_threshold4embryo=embryo_highpass_area_threshold,
                                                sigma_gaussian_smt=gaussian_smoothing_kernel,
                                                int_hist_bins=100,
                                                roi_mask=embryo_roi_mask,
                                                time_axis=time_axis,
                                                output_lowval=0,
                                                output_highval=255,
                                                output_dtype=np.uint8)
else:
    embryo_thresholded_timecourse = mask_embryo(ch1_timecourse,
                                                area_threshold4embryo=embryo_highpass_area_threshold,
                                                sigma_gaussian_smt=gaussian_smoothing_kernel,
                                                int_hist_bins=100,
                                                time_axis=time_axis,
                                                output_lowval=0,
                                                output_highval=255,
                                                output_dtype=np.uint8)

#Save the result with the correct data structure
embryo_thresholded_timecourse_saving_name = prefix_name + mask_sav_ing_name + ".ome.tif"




if time_axis==0:
    tifffile.imwrite(os.path.join(output_folder,embryo_thresholded_timecourse_saving_name), embryo_thresholded_timecourse, photometric='minisblack', metadata={"axes":'TYX'})
elif time_axis==1:
    tifffile.imwrite(os.path.join(output_folder,embryo_thresholded_timecourse_saving_name), embryo_thresholded_timecourse, photometric='minisblack', metadata={"axes":'YTX'})
elif time_axis==2:
    tifffile.imwrite(os.path.join(output_folder,embryo_thresholded_timecourse_saving_name), embryo_thresholded_timecourse, photometric='minisblack', metadata={"axes":'TYX'})
